In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

names = [
    "sys_time", "license_number", "lng", "lat", "gps_time", "EMPTY1", "speed",
    "direction", "car_status", "alarm_status", "EMPTY2", "EMPTY3",
    "license_color", "recorder_speed", "mileage", "height", "EMPTY4"
]

def notify(msg):
    import datetime
    channel="J0budaR2THarZw0OqS5O"
    notify_url=f"https://notify.run/{channel}"
    massage=f"{str(datetime.datetime.now())} | {msg}"
    os.system(f'curl {notify_url} -d "{massage}"')

%pwd

'/home/cseadmin/dz/TrafficFlowModel/data_process/taxi_clean'

In [2]:
# 获取taxi中所有文件，之后读取使用
root_path = os.walk("../../data/taxi")
all_files = []
for root,ds,fs in root_path:
    days = []
    for f in fs:
        full_path = os.path.join(root,f)
        days.append(full_path)
    if len(days)==0:
        continue
    all_files.append(days)
all_files

[['../../data/taxi/202006/2020-06-04/2020-06-04_6',
  '../../data/taxi/202006/2020-06-04/2020-06-04_0',
  '../../data/taxi/202006/2020-06-04/2020-06-04_7',
  '../../data/taxi/202006/2020-06-04/2020-06-04_2',
  '../../data/taxi/202006/2020-06-04/2020-06-04_8',
  '../../data/taxi/202006/2020-06-04/2020-06-04_5',
  '../../data/taxi/202006/2020-06-04/2020-06-04_4',
  '../../data/taxi/202006/2020-06-04/2020-06-04_9',
  '../../data/taxi/202006/2020-06-04/2020-06-04_1',
  '../../data/taxi/202006/2020-06-04/2020-06-04_3'],
 ['../../data/taxi/202006/2020-06-06/2020-06-06_6',
  '../../data/taxi/202006/2020-06-06/2020-06-06_8',
  '../../data/taxi/202006/2020-06-06/2020-06-06_2',
  '../../data/taxi/202006/2020-06-06/2020-06-06_7',
  '../../data/taxi/202006/2020-06-06/2020-06-06_3',
  '../../data/taxi/202006/2020-06-06/2020-06-06_9',
  '../../data/taxi/202006/2020-06-06/2020-06-06_5',
  '../../data/taxi/202006/2020-06-06/2020-06-06_0',
  '../../data/taxi/202006/2020-06-06/2020-06-06_4',
  '../../da

data check

---

In [3]:
df_temp=pd.read_csv("../../data/taxi/202006/2020-06-01/2020-06-01_0", names=names)

df_temp

,sys_time,license_number,lng,lat,gps_time,EMPTY1,speed,direction,car_status,alarm_status,EMPTY2,EMPTY3,license_color,recorder_speed,mileage,height,EMPTY4
0,2020-06-01 00:00:01,粤BD99912,113.98681,22.529696,2020-05-31 23:59:48,NaN,0,40,0,0,NaN,NaN,蓝色,0,2081590,0,0
1,2020-06-01 00:00:01,粤BDN1735,113.96201,22.536120,2020-05-31 23:59:49,NaN,0,0,0,0,NaN,NaN,蓝色,0,686220,0,0
2,2020-06-01 00:00:01,粤BDK8146,114.04288,22.598593,2020-05-31 22:22:57,NaN,0,173,0,0,NaN,NaN,蓝色,0,1894000,0,0
3,2020-06-01 00:00:01,粤BD84546,0.00000,0.000000,2020-05-31 23:59:49,NaN,0,0,0,32,NaN,NaN,蓝色,0,2484210,0,0
4,2020-06-01 00:00:01,粤BW7D07,0.00000,0.000000,2000-01-01 00:00:00,NaN,0,0,0,0,NaN,NaN,蓝色,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,2020-06-01 02:28:27,粤BD87346,113.92077,22.611652,2020-06-01 02:27:05,NaN,56,90,0,0,NaN,NaN,蓝色,56,3069870,0,0
9999996,2020-06-01 02:28:27,粤BD04142,114.13057,22.610834,2020-06-01 02:28:16,NaN,0,53,512,0,NaN,NaN,蓝色,0,4341550,0,0
9999997,2020-06-01 02:28:27,粤BD87432,113.81205,22.622503,2020-06-01 02:23:26,NaN,29,178,0,0,NaN,NaN,蓝色,29,0,0,0
9999998,2020-06-01 02:28:27,粤BD04878,113.98769,22.590467,2020-06-01 02:28:15,NaN,51,123,0,0,NaN,NaN,蓝色,51,922650,0,0


In [3]:
df_temp.sort_values(["license_number", "gps_time"], inplace=True)

df_temp[:100]

,sys_time,license_number,lng,lat,gps_time,EMPTY1,speed,direction,car_status,alarm_status,EMPTY2,EMPTY3,license_color,recorder_speed,mileage,height,EMPTY4
19259,2020-06-01 00:00:19,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
60555,2020-06-01 00:00:59,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
101973,2020-06-01 00:01:39,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
143621,2020-06-01 00:02:19,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
184634,2020-06-01 00:02:56,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3836710,2020-06-01 00:58:51,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
3877632,2020-06-01 00:59:26,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
3918249,2020-06-01 01:00:01,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
3959269,2020-06-01 01:00:36,粤B0334Y,113.90526,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0


In [26]:
last_row=df_temp.iloc[0]
index_list=[]
for index, row in df_temp.iterrows():
    if row.speed==0 and last_row.speed==0 and row.license_number==last_row.license_number and (row.lat!=last_row.lat or row.lng!=last_row.lng):
        index_list.append(index)
    last_row=row
    
    if len(index_list)>20:
        break

df_temp.loc[index_list]

,sys_time,license_number,lng,lat,gps_time,EMPTY1,speed,direction,car_status,alarm_status,EMPTY2,EMPTY3,license_color,recorder_speed,mileage,height,EMPTY4
674235,2020-06-01 00:10:43,粤B0C0P3,113.899100,22.566813,2020-06-01 00:10:35,NaN,0,67,512,0,NaN,NaN,蓝色,0,5686820,0,0
4321049,2020-06-01 01:05:54,粤B0C0P3,113.963646,22.543129,2020-06-01 01:05:45,NaN,0,4,0,0,NaN,NaN,蓝色,0,5687090,0,0
3122,2020-06-01 00:00:03,粤B0C0P5,114.057976,22.536942,2020-05-31 23:59:56,NaN,0,89,0,0,NaN,NaN,蓝色,0,6054070,0,0
4183603,2020-06-01 01:03:54,粤B0C0P5,114.019320,22.536503,2020-06-01 01:03:46,NaN,0,83,0,0,NaN,NaN,蓝色,0,6054310,0,0
4707511,2020-06-01 01:11:34,粤B0C0P5,114.019340,22.536283,2020-06-01 01:11:26,NaN,0,83,0,0,NaN,NaN,蓝色,0,6054310,0,0
7221428,2020-06-01 01:48:15,粤B0C0P5,114.000595,22.546928,2020-06-01 01:48:06,NaN,0,129,0,0,NaN,NaN,蓝色,0,6054370,0,0
7524351,2020-06-01 01:52:39,粤B0C0P5,113.992730,22.534624,2020-06-01 01:52:30,NaN,0,104,0,0,NaN,NaN,蓝色,0,6054390,0,0
7526577,2020-06-01 01:52:41,粤B0C0P5,113.992690,22.534658,2020-06-01 01:52:32,NaN,0,149,0,0,NaN,NaN,蓝色,0,6054390,0,0
7531427,2020-06-01 01:52:45,粤B0C0P5,113.992676,22.534750,2020-06-01 01:52:36,NaN,0,176,0,0,NaN,NaN,蓝色,0,6054390,0,0
8205639,2020-06-01 02:02:26,粤B0C0P5,114.047780,22.538462,2020-06-01 02:02:16,NaN,0,91,0,0,NaN,NaN,蓝色,0,6054450,0,0


In [ ]:
last_row=df_temp.iloc[0]
index_list=[]
for index, row in df_temp.iterrows():
    if row.speed==0 and last_row.speed==0 and row.license_number==last_row.license_number and row.lat==last_row.lat and row.lng==last_row.lng and row.gps_time!=last_row.gps_time:
        index_list.append(index)
    last_row=row
    
    if len(index_list)>20:
        break

df_temp.loc[index_list]

,sys_time,license_number,lng,lat,gps_time,EMPTY1,speed,direction,car_status,alarm_status,EMPTY2,EMPTY3,license_color,recorder_speed,mileage,height,EMPTY4
33597,2020-06-01 00:00:33,粤B0C0P3,113.949310,22.543388,2020-06-01 00:00:25,NaN,0,134,512,0,NaN,NaN,蓝色,0,5686740,0,0
43869,2020-06-01 00:00:43,粤B0C0P3,113.949310,22.543388,2020-06-01 00:00:35,NaN,0,134,512,0,NaN,NaN,蓝色,0,5686740,0,0
255942,2020-06-01 00:04:04,粤B0C0P3,113.919490,22.543232,2020-06-01 00:03:55,NaN,0,135,512,0,NaN,NaN,蓝色,0,5686770,0,0
277170,2020-06-01 00:04:23,粤B0C0P3,113.919490,22.543232,2020-06-01 00:04:15,NaN,0,135,512,0,NaN,NaN,蓝色,0,5686770,0,0
824020,2020-06-01 00:13:03,粤B0C0P3,113.902115,22.563807,2020-06-01 00:12:55,NaN,0,71,512,0,NaN,NaN,蓝色,0,5686820,0,0
856205,2020-06-01 00:13:33,粤B0C0P3,113.902115,22.563807,2020-06-01 00:13:25,NaN,0,71,512,0,NaN,NaN,蓝色,0,5686820,0,0
867545,2020-06-01 00:13:43,粤B0C0P3,113.902115,22.563807,2020-06-01 00:13:35,NaN,0,71,512,0,NaN,NaN,蓝色,0,5686820,0,0
878450,2020-06-01 00:13:53,粤B0C0P3,113.902115,22.563807,2020-06-01 00:13:45,NaN,0,71,512,0,NaN,NaN,蓝色,0,5686820,0,0
897454,2020-06-01 00:14:13,粤B0C0P3,113.902115,22.563807,2020-06-01 00:14:05,NaN,0,71,512,0,NaN,NaN,蓝色,0,5686820,0,0
1267349,2020-06-01 00:20:13,粤B0C0P3,113.909710,22.544931,2020-06-01 00:20:05,NaN,0,88,0,0,NaN,NaN,蓝色,0,5686850,0,0


In [6]:
last_row=df_temp.iloc[0]
index_list=[]
for index, row in df_temp.iterrows():
    if row.speed==0 and last_row.speed==0 and row.license_number==last_row.license_number and (row.lat!=last_row.lat or row.lng!=last_row.lng) and row.gps_time==last_row.gps_time:
        index_list.append(index)
    last_row=row
    
    if len(index_list)>5:
        break

df_temp.loc[index_list]

,sys_time,license_number,lng,lat,gps_time,EMPTY1,speed,direction,car_status,alarm_status,EMPTY2,EMPTY3,license_color,recorder_speed,mileage,height,EMPTY4


In [6]:
df_temp.loc[(df_temp["lng"]<=0) | (df_temp["lat"]<=0) | (df_temp["speed"]==0)]

,sys_time,license_number,lng,lat,gps_time,EMPTY1,speed,direction,car_status,alarm_status,EMPTY2,EMPTY3,license_color,recorder_speed,mileage,height,EMPTY4
0,2020-06-01 00:00:01,粤BD99912,113.98681,22.529696,2020-05-31 23:59:48,NaN,0,40,0,0,NaN,NaN,蓝色,0,2081590,0,0
1,2020-06-01 00:00:01,粤BDN1735,113.96201,22.536120,2020-05-31 23:59:49,NaN,0,0,0,0,NaN,NaN,蓝色,0,686220,0,0
2,2020-06-01 00:00:01,粤BDK8146,114.04288,22.598593,2020-05-31 22:22:57,NaN,0,173,0,0,NaN,NaN,蓝色,0,1894000,0,0
3,2020-06-01 00:00:01,粤BD84546,0.00000,0.000000,2020-05-31 23:59:49,NaN,0,0,0,32,NaN,NaN,蓝色,0,2484210,0,0
4,2020-06-01 00:00:01,粤BW7D07,0.00000,0.000000,2000-01-01 00:00:00,NaN,0,0,0,0,NaN,NaN,蓝色,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999980,2020-06-01 02:28:27,粤BDF4818,113.89893,22.563034,2020-06-01 02:28:14,NaN,0,151,0,0,NaN,NaN,蓝色,0,1854090,0,0
9999985,2020-06-01 02:28:27,粤BD90100,113.87854,22.593840,2020-06-01 01:11:21,NaN,0,124,0,0,NaN,NaN,蓝色,0,0,0,0
9999986,2020-06-01 02:28:27,粤BD02872,113.90954,22.587290,2020-06-01 02:28:17,NaN,0,48,0,0,NaN,NaN,蓝色,0,4736470,0,0
9999993,2020-06-01 02:28:27,粤BD88836,113.88458,22.586832,2020-06-01 02:27:46,NaN,0,159,512,0,NaN,NaN,蓝色,0,3297530,0,0


In [ ]:
df_temp.drop_duplicates(["lat", "lng"], inplace=True)

df_temp

,sys_time,license_number,lng,lat,gps_time,EMPTY1,speed,direction,car_status,alarm_status,EMPTY2,EMPTY3,license_color,recorder_speed,mileage,height,EMPTY4
19259,2020-06-01 00:00:19,粤B0334Y,113.905260,22.586212,2020-05-28 09:50:38,NaN,0,104,0,0,NaN,NaN,蓝色,0,0,0,0
19257,2020-06-01 00:00:19,粤B0994Y,114.017200,22.529259,2020-05-31 19:07:53,NaN,0,7,0,0,NaN,NaN,蓝色,0,7139,0,0
2744,2020-06-01 00:00:03,粤B0AE47,0.000000,0.000000,2000-01-01 00:00:00,NaN,0,0,0,0,NaN,NaN,蓝色,0,0,0,0
7802,2020-06-01 00:00:08,粤B0C0M7,114.164505,22.563550,2020-05-31 20:26:01,NaN,0,1,0,0,NaN,NaN,蓝色,0,5314570,0,0
15502,2020-06-01 00:00:15,粤B0C0P3,113.961080,22.543005,2020-05-31 23:58:25,NaN,60,144,512,0,NaN,NaN,蓝色,60,5686730,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7149085,2020-06-01 01:47:12,缌BDH3848,113.963610,22.523160,2020-06-01 01:47:01,NaN,66,134,0,0,NaN,NaN,蓝色,66,2751710,0,0
7277479,2020-06-01 01:49:04,缌BDH7901,114.217520,22.664618,2020-06-01 01:48:54,NaN,34,130,0,0,NaN,NaN,蓝色,34,1997700,0,0
6964178,2020-06-01 01:44:27,缌BDP1233,114.124840,22.560781,2020-06-01 01:44:17,NaN,6,87,0,0,NaN,NaN,蓝色,6,1681010,0,0
467676,2020-06-01 00:07:24,缌BDQ9220,114.113410,22.579279,2020-06-01 00:03:11,NaN,20,0,0,0,NaN,NaN,蓝色,20,0,0,0


In [5]:
df_temp["license_number"].nunique()

22547

In [4]:
all_rows=0
dirty_rows=0
for file in sum(all_files, []): # all_files.flatten()
    df_temp=pd.read_csv(file, names=names)
    all_rows+=len(df_temp)
    dirty_rows+=len(df_temp.loc[(df_temp["lng"]<=0) | (df_temp["lat"]<=0) | (df_temp["speed"]<=0) | (df_temp["speed"]>=120)])
    
print(all_rows)
print(dirty_rows)
print(dirty_rows/all_rows)

notify(f"dirty_rows/all_rows = {dirty_rows/all_rows}")

2545609441
1239008935
0.4867238921432017


TypeError: unsupported operand type(s) for |: 'str' and 'set'

---

In [ ]:
# 这两个日期是需要处理的文件日期开头以及结尾+1
open_day = pd.to_datetime("2020-06-01 00:00:00")
close_day = pd.to_datetime("2020-07-01 00:00:00")

# 将数据进行初步处理
# dest_path是中间结果，暂存到一个文件夹中，之后读取再进行处理
dest_path = "../../data/taxi_after_proc/intermediate/"
if not os.path.exists(dest_path):
    os.mkdir(dest_path)

all_files.sort()
for each_day in tqdm(all_files):
    each_day.sort()

    for each_time in tqdm(each_day):
        records = pd.read_csv(each_time, names=names)
        group = records.groupby("license_number")
        for each_lincense, each_record in group:
            posi = each_record[["lat", "lng", "gps_time", "speed"]]

            # 按gps时间排序，并去重
            posi["gps_time"] = pd.to_datetime(posi["gps_time"])
            posi.sort_values(by="gps_time", inplace=True, ascending=True)
            # posi = posi.drop_duplicates(subset="gps_time", keep="first")
            posi = posi.drop_duplicates(subset=["lat", "lng"], keep="first") # 改成按经纬度去重，尽量保留 speed=0 但是位置有变化的行，之后不再 drop speed=0

            posi = posi.loc[
                # 去除 GPS 0 值
                (posi["lat"] > 0) & (posi["lng"] > 0) &
                # 去除错误日期的记录
                (posi["gps_time"] > open_day) & (posi["gps_time"] < close_day) &
                # 保留速度 0~150 的记录 !这里为了尽量少删，保留速度=0，静止的行由上一步去重来删掉 2022-04-03
                # (posi["speed"] > 0) & 
                (posi["speed"] < 150)]

            # 去除数量过少的记录
            # if len(posi) < 30:
            #     continue

            # 将结果输出到 dest_path 中
            posi.to_pickle(dest_path + each_time[-7:] + each_lincense + ".pkl")
            
notify("Finish intermediate.")

 20%|██        | 2/10 [04:40<18:40, 140.06s/it]

In [ ]:
res_path="../../data/taxi_after_proc/clean202006/"
if not os.path.exists(res_path):
    os.mkdir(res_path)

files = os.listdir(dest_path)
car_set = set()

for file in files:
    car = file[7:-4]
    car_set.add(car)

days = []
for i in range(1, 31):
    days.append(str(i).zfill(2))

for car in tqdm(car_set):
    for day in days:
        if day=="19" or day=="20":
            continue
        df_all = pd.DataFrame()
        for i in range(11):
            try:
                df_all = df_all.append(
                    pd.read_pickle(
                        "../../data/taxi_after_proc/intermediate/06-{}_{}{}.pkl".
                        format(day, i, car)))
            except FileNotFoundError:
                continue
        if df_all.empty:
            continue
        df_all.sort_values(by="gps_time", inplace=True, ascending=True)
        df_all.drop_duplicates(subset=["lat", "lng"], keep="first", inplace=True)
        df_all.drop_duplicates(subset="gps_time", keep="first", inplace=True)
        df_all.to_pickle(
            "../../data/taxi_after_proc/clean202006/06-{}_{}.pkl".format(day, car))
        
notify("Finish 202006.")

  1%|          | 306/37486 [11:46<21:08:08,  2.05s/it]

In [ ]:
NUM_ROWS_ALL=2545609441
num_rows_remain=0
for processed_file in os.listdir(res_path):
    df_temp=pd.read_pickle(os.path.join(res_path, processed_file))
    num_rows_remain+=len(df_temp)
    
print(NUM_ROWS_ALL)
print(num_rows_remain)
print(num_rows_remain/NUM_ROWS_ALL)

notify(f"remaining_rows/all_rows = {num_rows_remain/NUM_ROWS_ALL}")

In [7]:
df1=pd.read_pickle("../../data/taxi_after_proc/intermediate/06-08_2粤BD05049.pkl")

df1

,lat,lng,gps_time,speed
30166,22.621632,114.156746,2020-06-08 02:17:32,0


In [6]:
df2=pd.read_pickle("../../data/taxi_after_proc/clean202006/06-08_粤BD05049.pkl")

df2

,lat,lng,gps_time,speed
11166,22.621640,114.156740,2020-06-07 23:56:21,0
9214635,22.621632,114.156746,2020-06-08 02:11:27,0
36576,22.621632,114.156746,2020-06-08 02:16:52,0
30166,22.621632,114.156746,2020-06-08 02:17:32,0
10804,22.621632,114.156746,2020-06-08 02:17:32,0
...,...,...,...,...
3410211,22.569810,114.117760,2020-06-08 23:55:54,63
3429444,22.572780,114.117880,2020-06-08 23:56:13,58
3458226,22.577087,114.116590,2020-06-08 23:56:43,60
3467801,22.578468,114.115870,2020-06-08 23:56:53,62
